In [ ]:
pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

In [ ]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
def tuner_model(hp):
  model = tf.keras.models.Sequential()

  hp_conv1 = hp.Int('conv1',min_value=32  , max_value = 64 ,step = 32)
  hp_conv2 = hp.Int('conv2',min_value=32  , max_value = 64 ,step = 32)
  hp_den1 = hp.Int('den1'  ,min_value=60  , max_value = 100 ,step = 20)
  hp_den2 = hp.Int('den2'  ,min_value=20  , max_value = 60 ,step = 20)
  hp_act1 = hp.Choice('activation',values = ['relu','tanh'])

  model.add(tf.keras.layers.Conv2D(filters = hp_conv1 , kernel_size =  (3,3),activation = hp_act1 , input_shape = (28,28,1)))
  model.add(tf.keras.layers.Conv2D(filters = hp_conv2 , kernel_size =  (3,3),activation = hp_act1 ))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(hp_den1,activation = hp_act1))
  model.add(tf.keras.layers.Dense(hp_den2,activation = hp_act1))
  model.add(tf.keras.layers.Dense(10,activation = 'softmax'))

  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001 , weight_decay=1e-5),
                loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics =['accuracy'])
  return model

In [ ]:
import keras_tuner as kt

tuner_ = kt.Hyperband(tuner_model,objective = 'val_accuracy' , max_epochs =  10 , factor = 3 , directory = 'keras_tuner',project_name=  'tuner_model')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss',patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience = 3)

In [ ]:
tuner_.search(x_train,y_train,epochs = 50 , validation_split=0.2 ,callbacks = [earlystop,reduce_lr])

Trial 30 Complete [00h 01m 24s]
val_accuracy: 0.9620000123977661

Best val_accuracy So Far: 0.9905833601951599
Total elapsed time: 00h 20m 39s


In [ ]:
model1 = tuner_.hypermodel.build(tuner_.get_best_hyperparameters(num_trials=1)[0])

In [ ]:
model1.fit(x_train,y_train,epochs = 50 , validation_split=0.2 ,callbacks = [earlystop,reduce_lr])

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0137 - accuracy: 0.9952 - val_loss: 0.0578 - val_accuracy: 0.9866 - lr: 0.0010
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0100 - accuracy: 0.9970 - val_loss: 0.0599 - val_accuracy: 0.9869 - lr: 0.0010
Epoch 3/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.0580 - val_accuracy: 0.9891 - lr: 0.0010
Epoch 4/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0082 - accuracy: 0.9977 - val_loss: 0.0712 - val_accuracy: 0.9862 - lr: 0.0010
Epoch 5/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.0573 - val_accuracy: 0.9893 - lr: 1.0000e-04
Epoch 6/50
1500/1500 [==============================] - 8s 6ms/step - loss: 1.7518e-04 - accuracy: 1.0000 - val_loss: 0.0587 - val_accuracy: 0.9894 - lr: 1.0000e-04
Epoch 7/50
1500/1500 [========================